# Nuclio - Process user location signal

## Setup the environment

In [1]:
# nuclio: ignore
import nuclio

In [30]:
%%nuclio cmd
pip install v3io_frames
pip install v3io==0.1.1
pip install requests
pip install pandas
# pip install json
# pip install time

  Running setup.py bdist_wheel for v3io ... done
  Stored in directory: /igz/.cache/pip/wheels/16/06/30/71478da3b7ae64df011a3466bcea2653a35ba3c5aa43ca1331
Successfully built v3io
  Found existing installation: v3io 0.0.2
    Uninstalling v3io-0.0.2:
      Successfully uninstalled v3io-0.0.2


In [45]:
%%nuclio env
COSTUMERS_TABLE=customers
STORES_TABLE=stores
PREDICTIONS_TABLE=predictions

%nuclio: setting 'COSTUMERS_TABLE' environment variable
%nuclio: setting 'STORES_TABLE' environment variable
%nuclio: setting 'PREDICTIONS_TABLE' environment variable


In [46]:
import json
import os
import requests
import time

# Data handling
import pandas as pd

# DB
import v3io
import v3io.dataplane
import v3io.logger
import v3io_frames as v3f

In [62]:
def init_context(context):
    # DB Contexts
    
    v3c = v3io.dataplane.Context(v3io.logger.Logger('DEBUG')).new_session().new_container('bigdata')
    setattr(context, 'v3c', v3c)
    
    v3c_frames = v3f.Client('http://' + os.environ['V3IO_FRAMESD'])
    setattr(context, 'v3f', v3c_frames)
    
    
    # DB Tables
    
    customers_table = os.environ['COSTUMERS_TABLE']
    setattr(context, 'customers', customers_table)
    
    stores_table = os.environ['STORES_TABLE']
    setattr(context, 'stores', stores_table)
    
    predictions_table = os.environ['PREDICTIONS_TABLE']
    setattr(context, 'predictions', predictions_table)
    
    
    # Prediction server
    
    prediction_server = os.getenv('PREDICTION_SERVER')
    setattr(context, 'prediction_server', prediction_server)

In [63]:
# nuclio: ignore
init_context(context)

In [61]:
def handler(context, event):
    
    # Extract data from event
    try:
        jevent = json.loads(event.body)
    except Exception as e:
        context.logger.warn(f'Bad input: {e}')
        jevent = {"location": '01', "user_id": '02'}
    user_id = jevent['user_id']
    location = jevent['location']

    
    # Update the customer's new location
    update_customer_location(context, user_id, location)

    
    # Check if store location exists
    is_store, store = is_user_in_store(location)
    if not is_store:
        return _build_response(context, json.dumps({'intersection': False, 'is_store': is_store, 'store': store}), 200)

    resp = store

    
    # Get prediction from server
    prediction_url = "{0}{1}".format(context.prediction_server, user_id)
    predictions = requests.get(prediction_url)
    try:
        prediction_result = pd.DataFrame(predictions.json())
    except:
        prediction_result = {'prediction': 'None', 'product_id': 0}
        
        
    # Save prediction
    save_predictions(context, user_id, prediction_result)

    
    # Return response
    resp = {
        'user_id': user_id,
        'location': location,
        'is_store': is_store,
        'predictions': prediction_result,
    }

    return _build_response(context, json.dumps(resp), 200)

def _build_response(context, body, status_code):
    return context.Response(body=body,
                            headers={},
                            content_type='application/json',
                            status_code=status_code)

def is_user_in_store(context, store_location: str) -> bool:
    store = v3io.dataplane.GetItemInput(v3io.common.helpers.url_join(context.stores, store_location))
    store = context.v3c.get_item(store)
    if store.status_code == 200:
        return True, store.output.item
    else:
        return False, {}

def get_customer(context, customer_id: str):
    customer = v3io.dataplane.PutItemInput(v3io.common.helpers.url_join(context.customers, customer_id))
    customer = context.v3c.put_item(customer)
    
    return customer.output.item

def update_customer_location(context, customer_id: str, location: str):
    customer = v3io.dataplane.UpdateItemInput(v3io.common.helpers.url_join(context.customers, customer_id), 
                                              {'location': location})
    customer = context.v3c.update_item(customer)
    
    return customer.item

def save_predictions(context, customer_id: str, prediction: pd.DataFrame):
    context.v3f.write('tsdb', context.predictions, prediction)